In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path

In [ ]:
commission_multiple = {
    '10032518': 1.1,
    '12200707': 1.3
}

size_dict = {
    'IF': 300,
    'IH': 300,
    'IC': 200
}

def strip_digit(symbol: str):
    res = ""
    for char in symbol:
        if not char.isdigit():
            res += char
        else:
            break
    return res

def calc_trade(df: pd.DataFrame, account_type: str):
    df_sel = df[df['account_type'] == account_type].copy()
    df_sel['size'] = df_sel['symbol'].map(strip_digit).map(size_dict)
    df_sel['turnover'] = df_sel['price'] * df_sel['volume'] * df_sel['size']
    df_sel['com_multiple'] = df_sel['account_id'].map(commission_multiple)
    df_sel['comission'] = df_sel['turnover'] * df_sel['com_multiple'] * (0.23 / 10000)
    
    df_sel_long = df_sel[df_sel['direction'] == "多"]
    df_sel_short = df_sel[df_sel['direction'] == "空"]
    
    d = {}
    d['date'] = df_sel.iloc[0]['date']
    d['account'] = df_sel.iloc[0]['account_id']
    d['comission'] = df_sel['comission'].sum()
    d['long_volume'] = df_sel_long['volume'].sum()
    d['long_turnover'] = df_sel_long['turnover'].sum()
    d['long_cost'] = sum(df_sel_long['price'] * df_sel_long['volume']) / d['long_volume']
    d['short_volume'] = df_sel_short['volume'].sum()
    d['short_turnover'] = df_sel_short['turnover'].sum()
    d['short_cost'] = sum(df_sel_short['price'] * df_sel_short['volume']) / d['short_volume']
    d['trade_pnl'] = d['short_turnover'] - d['long_turnover']
    d['net_pnl'] = d['trade_pnl'] - d['comission']
    d['all_volume'] = d['long_volume'] + d['short_volume']
    
    df = pd.DataFrame([d])
#     df.set_index('date', inplace=True)
    return df

def stats_trade_by_date(file):
    df = pd.read_csv(file, parse_dates=[0])
    df[['account_id', 'tradeid']] = df[['account_id', 'tradeid']].astype('str')
    
    source_res = calc_trade(df, 'source')
    target_res = calc_trade(df, 'target')
    
    df_res = pd.merge(source_res, target_res, how='outer', on='date', suffixes=('_source', '_target'))
    df_res['net_pnl_diff'] = df_res['net_pnl_source'] - df_res['net_pnl_target']
    df_res['trade_pnl_diff'] = df_res['trade_pnl_source'] - df_res['trade_pnl_target']
    df_res['long_diff'] = df_res['long_cost_target'] - df_res['long_cost_source']
    df_res['short_diff'] = df_res['short_cost_source'] - df_res['short_cost_target']
    df_res['trade_diff'] = (df_res['long_diff'] + df_res['short_diff']) / 2
    
    return df_res

In [ ]:
folder = Path(r'D:\work')
files = folder.glob('*.csv')
for file in files:
    print(file, type(file))

In [ ]:
file = Path(r'D:\work\trade_20200730.csv')
df = stats_trade_by_date(file)

In [ ]:
df